<a href="https://colab.research.google.com/github/lintangiqhtiar/rec-sys-based-llm/blob/main/sentimen_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [1]:
import pandas as pd
import re

#Input Dataset

In [2]:
df=pd.read_csv('/content/playstore_reviews (1).csv')

#Cleaning

In [3]:
df_prepro=df.copy()

In [4]:
def clean_review(text):
    if not text: return ""
    text = re.sub(r'<.*?>', '', text)  # Hapus HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Hapus simbol
    text = re.sub(r'\s+', ' ', text).strip()  # Hapus spasi berlebih
    return text.lower()  # Ubah huruf kecil (case folding)

In [5]:
df_prepro.drop(['Rating', 'User', 'Date', 'Reply', 'sentimen'], axis=1, inplace=True)

In [6]:
df_prepro["Review"] = df_prepro["Review"].apply(clean_review)  # Bersihkan Review

#Normalisasi

In [7]:
# normalisasi
normalisasi_kata_df = pd.read_csv('/content/normalisasi-new.csv')
normalisasi_kata_dict = dict(zip(normalisasi_kata_df['before'], normalisasi_kata_df['after']))

def normalisasi(text):
    return [normalisasi_kata_dict.get(word, word) for word in text]

In [8]:
#normalisasi
df_prepro['Review'] = df_prepro['Review'].apply(lambda x: normalisasi(x.split()))

In [9]:
#menggabungkan value Review menjadi 1 string
for i in range(len(df_prepro)):
  df_prepro['Review'][i]=' '.join(df_prepro['Review'][i])

<ipython-input-9-290b61cef843>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_prepro['Review'][i]=' '.join(df_prepro['Review'][i])


#Sentiment analysisi with pretrain bert model

In [10]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")
print(sentiment_analysis("Saya sangat menyukai aplikasinya."))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/476k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'Positive', 'score': 0.9998088479042053}]


In [11]:
df['sentimen']=df['Review'].apply(lambda x : sentiment_analysis(x))

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [12]:
df

,Review,Rating,User,Date,Reply,sentimen
0,Saya tidak menemukan menu lanjutan untuk perpa...,3,Pengguna Google,2024-12-12 03:36:24,NaN,"[{'label': 'Neutral', 'score': 0.9996390342712..."
1,harusnya proses itu setelah data dikirim baru ...,1,Pengguna Google,2024-12-12 03:22:08,NaN,"[{'label': 'Negative', 'score': 0.996880054473..."
2,Sangat bagus dan proses nya cepat,5,Pengguna Google,2024-12-12 03:16:52,NaN,"[{'label': 'Positive', 'score': 0.999757349491..."
3,"Ini aplikasinya kenapa sih,kok mau buat SIM ba...",2,Pengguna Google,2024-12-12 03:11:16,NaN,"[{'label': 'Negative', 'score': 0.999675512313..."
4,"Aplikasi Payah, duit udah dibayar tapi muncul ...",1,Pengguna Google,2024-12-12 02:50:28,NaN,"[{'label': 'Negative', 'score': 0.999853253364..."
...,...,...,...,...,...,...
33258,Mantaps,5,Pengguna Google,2022-01-01 02:31:50,Terimakasih telah memberikan penilaian baik un...,"[{'label': 'Positive', 'score': 0.999678134918..."
33259,"Lanjut perpanjangan SIM secara manual, lewat o...",2,Pengguna Google,2022-01-01 02:28:54,NaN,"[{'label': 'Neutral', 'score': 0.9721829295158..."
33260,Knp katanya layanan blom tersedia ada masalah ...,1,Pengguna Google,2022-01-01 02:06:50,Saat ini Aplikasi Digital Korlantas POLRI hany...,"[{'label': 'Negative', 'score': 0.614702522754..."
33261,"Bagusss, beberapa kali buat lancar",5,Pengguna Google,2022-01-01 01:57:09,Terimakasih telah memberikan penilaian baik un...,"[{'label': 'Positive', 'score': 0.999539256095..."


In [13]:
df_prepro['sentimen']=df_prepro['Review'].apply(lambda x : sentiment_analysis(x))

In [14]:
df_prepro

,Review,sentimen
0,saya tidak menemukan menu lanjutan untuk perpa...,"[{'label': 'Neutral', 'score': 0.9996390342712..."
1,harusnya proses itu setelah data dikirim baru ...,"[{'label': 'Negative', 'score': 0.988939702510..."
2,sangat bagus dan proses nya cepat,"[{'label': 'Positive', 'score': 0.999757349491..."
3,ini aplikasinya kenapa sihkok mau buat sim bar...,"[{'label': 'Negative', 'score': 0.994383811950..."
4,aplikasi payah duit sudah dibayar tapi muncul ...,"[{'label': 'Negative', 'score': 0.999813139438..."
...,...,...
33258,mantaps,"[{'label': 'Positive', 'score': 0.999678134918..."
33259,lanjut perpanjangan sim secara manual lewat on...,"[{'label': 'Neutral', 'score': 0.9986758828163..."
33260,kenapa katanya layanan belum tersedia ada masa...,"[{'label': 'Negative', 'score': 0.987647891044..."
33261,bagus beberapa kali buat lancar,"[{'label': 'Positive', 'score': 0.999572813510..."


In [17]:
df_prepro['label']=df_prepro['sentimen'].apply(lambda x: x[0]['label'])
df_prepro['confidence']=df_prepro['sentimen'].apply(lambda x: x[0]['score'])

In [19]:
df_prepro.drop('sentimen', axis=1, inplace=True)

In [20]:
df_prepro

,Review,label,confidence
0,saya tidak menemukan menu lanjutan untuk perpa...,Neutral,0.999639
1,harusnya proses itu setelah data dikirim baru ...,Negative,0.988940
2,sangat bagus dan proses nya cepat,Positive,0.999757
3,ini aplikasinya kenapa sihkok mau buat sim bar...,Negative,0.994384
4,aplikasi payah duit sudah dibayar tapi muncul ...,Negative,0.999813
...,...,...,...
33258,mantaps,Positive,0.999678
33259,lanjut perpanjangan sim secara manual lewat on...,Neutral,0.998676
33260,kenapa katanya layanan belum tersedia ada masa...,Negative,0.987648
33261,bagus beberapa kali buat lancar,Positive,0.999573


In [25]:
neg_count = (df_prepro['label']=='Negative').sum()
neu_count = (df_prepro['label']=='Neutral').sum()
pos_count = (df_prepro['label']=='Positive').sum()

In [27]:
print("Jumlah label Negative: ", neg_count)
print("Jumlah label Neutral: ", neu_count)
print("Jumlah label Positive: ", pos_count)

Jumlah label Negative:  11856
Jumlah label Neutral:  8283
Jumlah label Positive:  13124


In [30]:
#melihat baris yang memiliki label heutral
df_prepro[df_prepro['label']=='Neutral']

,Review,label,confidence
0,saya tidak menemukan menu lanjutan untuk perpa...,Neutral,0.999639
6,semua layanan belum tersedia kan wkwk,Neutral,0.999070
15,tingkatkan,Neutral,0.987647
17,mau buat sim baru apakah nya belum siap,Neutral,0.999522
19,saya tidak bisa mendapatkan cara atau menu bua...,Neutral,0.999696
...,...,...,...
33247,sim hilang masih hafal nomor kartu untuk bukti...,Neutral,0.997337
33249,tidak bisa melanjutkan proses karena nama ibu ...,Neutral,0.562984
33253,kenapa masalah daftar aplikasi dengan nomer ti...,Neutral,0.999143
33259,lanjut perpanjangan sim secara manual lewat on...,Neutral,0.998676


In [41]:
df_sentiment = df_prepro.copy()
df_sentiment.loc[df_sentiment['label']=='Neutral', 'label'] = 'Negative'

In [42]:
df_sentiment.head()

,Review,label,confidence
0,saya tidak menemukan menu lanjutan untuk perpa...,Negative,0.999639
1,harusnya proses itu setelah data dikirim baru ...,Negative,0.988940
2,sangat bagus dan proses nya cepat,Positive,0.999757
3,ini aplikasinya kenapa sihkok mau buat sim bar...,Negative,0.994384
4,aplikasi payah duit sudah dibayar tapi muncul ...,Negative,0.999813


In [43]:
df_sentiment['label'].value_counts()

,count
label,
Negative,20139
Positive,13124


In [45]:
df_sentiment.to_csv('sentiment_analysis.csv', index=False)
df_prepro.to_csv('prepro_sentiment_analysis.csv', index=False)